In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm 

from shapely.geometry import LineString
from shapely.ops import nearest_points

from traffic.core import Traffic
from traffic.data import airports
from traffic.data import navaids

/tmp/ipykernel_8423/2212634332.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
t_ga = Traffic.from_file("../deep_traffic_generation/data/training_datasets/ga_LSZH_14.pkl")
t_to = Traffic.from_file("../deep_traffic_generation/data/training_datasets/to_LSZH_16.pkl")

To do list:
- Create a function that associates randomly points from 2 different Trafficdatasets (one to, one ga | See TrafficDatasetPairs)
- Create a flag if the pair is crossing or not (or if they have a min distance inferior to 1Nm and 1000 ft)

In [3]:
import random

f_id_ga = t_ga.flight_ids
f_id_to = t_to.flight_ids

pairs = [(f1,f2) for f1 in f_id_to for f2 in f_id_ga]
choice_id = [random.choice(pairs) for _ in range(10000)]

In [4]:
from ipyleaflet import Map, basemaps
from ipywidgets import Layout
from traffic.data import airports

map_ = Map(
    center=airports["LSZH"].latlon,
    zoom=10,
    basemap=basemaps.Stamen.Terrain,
    layout=Layout(width="100%", height="1000px"),
)

map_.add_layer(t_to[choice_id[0][0]])
map_.add_layer(t_ga[choice_id[0][1]])

map_

Map(center=[47.458056, 8.548056], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [5]:
from shapely.geometry import Point, MultiPoint

crossings = {}
for i in tqdm(choice_id):
    a = t_to[i[0]]
    b = t_ga[i[1]]
    isect = a.linestring.intersection(b.linestring)
    
    if isinstance(isect, MultiPoint):
        crossings[i] = len(isect)
        
    elif isinstance(isect, Point):
        crossings[i] = 1
    
    else:
        crossings[i] = 0
        

  0%|          | 0/10000 [00:00<?, ?it/s]

/tmp/ipykernel_8423/4017365723.py:10: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  crossings[i] = len(isect)


In [15]:
test = pd.Series(crossings)
test.value_counts()

2    7495
0    2440
3      30
4      16
1      12
6       1
dtype: int64

In [9]:
#BTI7PY_109 is not smooth at the second crossing point
#SWR18_4042 is not smooth

#ATTENTION: On a des N+chiffre qui représentent des petits avions

isect = t_to['THA971_7256'].linestring.intersection(t_ga['SWR1663_578'].linestring)
for t in isect.geoms:
    print(t)

POINT Z (8.626060567768787 47.416393027423446 8834.574540755399)
POINT Z (8.68179533167382 47.411835544670346 9339.633411993918)
POINT Z (8.634503852467974 47.4118141405333 9336.14153240541)
POINT Z (8.688197249697913 47.46255734388703 4861.73196332977)
POINT Z (8.693799432928493 47.43686118446189 7439.970776575676)
POINT Z (8.612389749972015 47.426391362601656 3844.004700618021)


In [10]:
map_ = Map(
    center=airports["LSZH"].latlon,
    zoom=10,
    basemap=basemaps.Stamen.Terrain,
    layout=Layout(width="100%", height="1000px"),
)

map_.add_layer(t_to['THA971_7256'])
map_.add_layer(t_ga['SWR1663_578'])

map_

Map(center=[47.458056, 8.548056], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [61]:
isect = a.linestring.intersection(b.linestring)
lon,lat = nearest_points(a.linestring, isect)[0].coords.xy

In [64]:
isinstance(isect, MultiPoint)

True

In [26]:
isect.x

AttributeError: 'MultiPoint' object has no attribute 'x'